# Scraping Wikipedia about Stolpersteine in Hamburg (Germany, Europe) – Part 2

#### In the first draft this project will be about the city of Hamburg where I live. In part 1 I started in my own neighborhood in the borough Wilhelmsburg. Now I will apply my scraper to the whole city
#### Later on I want to expand my project to every city of Germany and every other place where you can find them

In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

## 1. Get urls for every "Stadtteil" (borough)

In [48]:
url_hamburg = "https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg"
html = requests.get(url_hamburg).text

In [49]:
# This code will not help me here because the tables with text I want to change are not on the first website

#html = html.replace("<br /><b>", " ")
#html = html.replace("</b><br />", " ")
#html = html.replace("<br />", " ")

In [50]:
soup = BeautifulSoup(html)

## 2. Select tables for each "Bezirk" (districts) and its "Stadtteile" (boroughs)

#### Hamburg has 7 districs and 104 boroughs

In [51]:
altona = soup.select("table.wikitable")[0]
bergedorf = soup.select("table.wikitable")[1]
eimsbüttel = soup.select("table.wikitable")[2]
hamburg_mitte = soup.select("table.wikitable")[3]
hamburg_nord = soup.select("table.wikitable")[4]
harburg = soup.select("table.wikitable")[5]
wandsbek = soup.select("table.wikitable")[6]

In [52]:
tables = [altona, bergedorf, eimsbüttel, hamburg_mitte, hamburg_nord, harburg, wandsbek]

In [53]:
## Not in all of the boroughs are stones - I could also get the names of the NaN borough from the table

all_boroughs = []
for table in tables:
    for cell in table.select("a"):
         all_boroughs.append(cell.text.strip())
        
len(all_boroughs)

80

## Got complete list of urls but had problems with results later because columns are different. Go on with each Bezirk

## (1) Altona

In [54]:
## len should be 13

boroughs_altona = []

for cell in altona.select("a"):
    stadtteil = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    boroughs_altona.append({
        'Bezirk' : 'Altona',
        'Stadtteil' : stadtteil,
        'URL' : url
    })    

In [55]:
boroughs_altona

for b in boroughs_altona:
    url = b['URL'] 
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    b['Table'] = soup.select("table.wikitable")[0]
    b_table = b['Table']

print(b_table.text[:100])



Adresse

Person(en)

Inschrift

Bilder

Anmerkung


Altonaer Straße 8

Minna Littmann geb. Harriso


In [56]:

header_el = b_table.select("tr")[0]
columns = []
for cell in header_el.select("th"):
    columns.append(cell.text.strip())
columns.append('Bezirk')
columns.append('Stadtteil')
columns

['Adresse',
 'Person(en)',
 'Inschrift',
 'Bilder',
 'Anmerkung',
 'Bezirk',
 'Stadtteil']

In [57]:
### here the loop should get the name of the 'Stadtteil' matching to the 'Table' – but it only takes the last one

for b in boroughs_altona:
    b['Table'] = b_table
    row_els = b_table.select("tr")[1:]
    rows = []
    for row in row_els:
        cells = []
        for cell in row.select("td"):
            cells.append(cell.text.strip())
        cells.append(b['Bezirk'])    
        cells.append(b['Stadtteil'])
        rows.append(cells)   
rows[:2]        

[['Altonaer Straße 8',
  'Minna Littmann geb. Harrison',
  'Hier wohnte Minna Littmann geb. Harrison Jg. 1869 deportiert 1942 Theresienstadt ermordet in Auschwitz',
  '',
  'Eintrag auf stolpersteine-hamburg.deAuf stolpersteine-hamburg.de unter Eppendorfer Landstraße 12 geführt.',
  'Altona',
  'Sternschanze'],
 ['Altonaer Straße 8',
  'Simon Mayer Littmann',
  'Hier wohnte Simon Mayer Littmann Jg. 1870 deportiert 1942 Theresienstadt ermordet 31.8.1942',
  '',
  'Eintrag auf stolpersteine-hamburg.deAuf stolpersteine-hamburg.de unter Eppendorfer Landstraße 12 geführt.',
  'Altona',
  'Sternschanze']]

In [58]:
## count should be 106

df = pd.DataFrame(rows, columns=columns)
df.describe()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil
count,106,106,106,106,106,106,106
unique,44,106,106,1,7,1,1
top,Juliusstraße 35,Minna Littmann geb. Harrison,Hier wohnte Minna Littmann geb. Harrison Jg. 1...,,Eintrag auf stolpersteine-hamburg.de,Altona,Sternschanze
freq,7,1,1,106,99,106,106


In [59]:
df.head()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil
0,Altonaer Straße 8,Minna Littmann geb. Harrison,Hier wohnte Minna Littmann geb. Harrison Jg. 1...,,Eintrag auf stolpersteine-hamburg.deAuf stolpe...,Altona,Sternschanze
1,Altonaer Straße 8,Simon Mayer Littmann,Hier wohnte Simon Mayer Littmann Jg. 1870 depo...,,Eintrag auf stolpersteine-hamburg.deAuf stolpe...,Altona,Sternschanze
2,Bartelsstraße 9,Charlotte Frenkel geb. Tugendhaft,Hier wohnte Charlotte Frenkel geb. Tugendhaft ...,,Eintrag auf stolpersteine-hamburg.de,Altona,Sternschanze
3,Bartelsstraße 9,Etti Frenkel,Hier wohnte Etti Frenkel Jg. 1936 'Polenaktion...,,Eintrag auf stolpersteine-hamburg.de,Altona,Sternschanze
4,Bartelsstraße 9,Pinkus Frenkel,Hier wohnte Pinkus Frenkel Jg. 1903 'Polenakti...,,Eintrag auf stolpersteine-hamburg.de,Altona,Sternschanze


In [60]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for row in row_els:
    cells = []
    geo = []
    for cell in row.select("a"):
        cells.append(cell["href"])  
    for cell in cells:
        parsed_url = urlparse(cell)[4]
        geo.append(parsed_url)  
    for g in geo:
        if "pagename" in g:
            Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Sternschanze&language=de&params=53.563810_N_9.960845_E_region:DE-HH_type:landmark&title=Minna+Littmann+geb.+Harrison',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Sternschanze&language=de&params=53.563810_N_9.960845_E_region:DE-HH_type:landmark&title=Simon+Mayer+Littmann']

In [61]:
df['Geohack_url'] = Geohack

df['params'] = df['Geohack_url'].str.extract("params=(.*)region", expand=False)
df['lat'] = df['params'].str.extract("(.*)_N", expand=False)
df['lon'] = df['params'].str.extract("N_(.*)_E", expand=False)

df.head()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil,Geohack_url,params,lat,lon
0,Altonaer Straße 8,Minna Littmann geb. Harrison,Hier wohnte Minna Littmann geb. Harrison Jg. 1...,,Eintrag auf stolpersteine-hamburg.deAuf stolpe...,Altona,Sternschanze,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.563810_N_9.960845_E_,53.563810,9.960845
1,Altonaer Straße 8,Simon Mayer Littmann,Hier wohnte Simon Mayer Littmann Jg. 1870 depo...,,Eintrag auf stolpersteine-hamburg.deAuf stolpe...,Altona,Sternschanze,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.563810_N_9.960845_E_,53.563810,9.960845
2,Bartelsstraße 9,Charlotte Frenkel geb. Tugendhaft,Hier wohnte Charlotte Frenkel geb. Tugendhaft ...,,Eintrag auf stolpersteine-hamburg.de,Altona,Sternschanze,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.561507_N_9.965130_E_,53.561507,9.965130
3,Bartelsstraße 9,Etti Frenkel,Hier wohnte Etti Frenkel Jg. 1936 'Polenaktion...,,Eintrag auf stolpersteine-hamburg.de,Altona,Sternschanze,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.561507_N_9.965130_E_,53.561507,9.965130
4,Bartelsstraße 9,Pinkus Frenkel,Hier wohnte Pinkus Frenkel Jg. 1903 'Polenakti...,,Eintrag auf stolpersteine-hamburg.de,Altona,Sternschanze,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.561507_N_9.965130_E_,53.561507,9.965130


In [62]:
#df.to_csv('stolpersteine_altona.csv', index = False)

## (2) Bergedorf

In [63]:
## len should be 2, but the tables are the same (both boroughs are in one table, have the same link)

boroughs_bergedorf = []

for cell in bergedorf.select("a"):
    stadtteil = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    boroughs_bergedorf.append({
        'Bezirk' : 'Bergedorf',
        'Stadtteil' : stadtteil,
        'URL' : url
    })   

print(boroughs_bergedorf)

[{'Bezirk': 'Bergedorf', 'Stadtteil': 'Bergedorf', 'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Bergedorf'}, {'Bezirk': 'Bergedorf', 'Stadtteil': 'Lohbrügge', 'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Bergedorf'}]


In [64]:
boroughs_bergedorf

for b in boroughs_bergedorf:
    url = b['URL'] 
    html = requests.get(b['URL']).text
    soup = BeautifulSoup(html)
    b['Table'] = soup.select("table.wikitable")[0]
    b_table = b['Table']

print(b_table.text[:100])



Adresse

Name

Leben

Jahr

Bilder


Alte Holstenstraße 61 

Ernst Tichauer

Ernst Siegfried Ticha


In [65]:
header_el = b_table.select("tr")[0]
columns = []
for cell in header_el.select("th"):
    columns.append(cell.text.strip())
columns.append('Bezirk')
columns.append('Stadtteil')
columns

['Adresse', 'Name', 'Leben', 'Jahr', 'Bilder', 'Bezirk', 'Stadtteil']

In [66]:
for b in boroughs_bergedorf:
    b['Table'] = b_table
    row_els = b_table.select("tr")[1:]
    rows = []
    for row in row_els:
        cells = []
        for cell in row.select("td"):
            cells.append(cell.text.strip())
        cells.append(b['Bezirk'])    
        cells.append(b['Stadtteil'])
        rows.append(cells)   
rows[:2]        

[['Alte Holstenstraße 61',
  'Ernst Tichauer',
  'Ernst Siegfried Tichauer (* 8. Oktober 1888 in Thorn, Westpreußen; † 1943?) war ein promovierter jüdischer Zahnarzt. Seine Frau assistierte in seiner Praxis in der Holstenstraße. Das Schicksal des Ehepaars in der Zeit des Nationalsozialismus ist exemplarisch für die systematische rassistische Diffamierung, Ausgrenzung und Verfolgung deutscher jüdischer Mitbürger. Anfang 1933 brachten NSDAP-Mitglieder an Tichauers Praxis ein Schild: „Wer zum Juden geht…“ an. 1935 trat das Ehepaar aus Protest gegen die Drangsalierungen in die Jüdische Gemeinde ein. Nach den Novemberpogromen 1938 musste Tichauer zwangsweise sein Haus zum halben Preis verkaufen, 1939 wurde ihm die zahnärztliche Approbation entzogen. Im Februar 1939 war er gezwungen, zusammen mit seiner Frau ein „Judenhaus“ in Hamburg-Harvestehude zu beziehen. Ab September 1941 musste das Ehepaar den Judenstern tragen. Am 8. November 1941 wurde das Ehepaar ins Ghetto Minsk deportiert. Ein le

In [67]:
## has extra column for year
## count should be 34

df = pd.DataFrame(rows, columns=columns)
df.describe()

,Adresse,Name,Leben,Jahr,Bilder,Bezirk,Stadtteil
count,34,34,34,34,34,34,34
unique,26,34,34,6,34,1,1
top,Ernst-Mantius-Straße 5,Ernst Tichauer,Ernst Siegfried Tichauer (* 8. Oktober 1888 in...,2003,Stolpersteine für Ernst und Ellie Tichauer Woh...,Bergedorf,Lohbrügge
freq,5,1,1,21,1,34,34


In [68]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for row in row_els:
    cells = []
    geo = []
    for cell in row.select("a"):
        cells.append(cell["href"])  
    for cell in cells:
        parsed_url = urlparse(cell)[4]
        geo.append(parsed_url)  
    for g in geo:
        if "pagename" in g:
            Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Bergedorf&language=de&params=53.4900926_N_10.2090303_E_region:DE-HH_type:landmark&title=Dr.+Ernst+Tichauer%2C+Alte+Holstenstra%C3%9Fe+61',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Bergedorf&language=de&params=53.4900724_N_10.2090492_E_region:DE-HH_type:landmark&title=Ellie+Tichauer+geb.+Rosenthal%2C+Alte+Holstenstra%C3%9Fe+61']

In [69]:
df['Geohack_url'] = Geohack

df['params'] = df['Geohack_url'].str.extract("params=(.*)region", expand=False)
df['lat'] = df['params'].str.extract("(.*)_N", expand=False)
df['lon'] = df['params'].str.extract("N_(.*)_E", expand=False)

df.head()

,Adresse,Name,Leben,Jahr,Bilder,Bezirk,Stadtteil,Geohack_url,params,lat,lon
0,Alte Holstenstraße 61,Ernst Tichauer,Ernst Siegfried Tichauer (* 8. Oktober 1888 in...,2003,Stolpersteine für Ernst und Ellie Tichauer Woh...,Bergedorf,Lohbrügge,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4900926_N_10.2090303_E_,53.4900926,10.2090303
1,Alte Holstenstraße 61,Ellie Tichauer,"Ellie Tichauer (* 6. November 1887 in Berlin, ...",2003,Stolperstein für Ellie Tichauer,Bergedorf,Lohbrügge,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4900724_N_10.2090492_E_,53.4900724,10.2090492
2,August-Bebel-Straße 1,Mary Dobrzinsky,Mary Dobrzinski (* 1. Oktober 1880 in Bergedor...,2003,Stolperstein für Mary Dobrzinsky\nDie August-B...,Bergedorf,Lohbrügge,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4852604_N_10.2184875_E_,53.4852604,10.2184875
3,August-Bebel-Straße 47,Vera Krebs,,2024,Stolperstein für Vera Krebs,Bergedorf,Lohbrügge,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.485086_N_10.223984_E_,53.485086,10.223984
4,August-Bebel-Straße 103,Claus Beeck,Claus Beeck (* 28. Januar 1940 in Bergedorf; †...,2013,"Stolperstein für Claus Beeck\nDie Gebäude, in ...",Bergedorf,Lohbrügge,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4842576_N_10.2290653_E_,53.4842576,10.2290653


In [70]:
#df.to_csv('stolpersteine_bergedorf.csv', index = False)

## (3) Eimsbüttel

## Incomplete - Here the scraper does not work but why???

In [74]:
boroughs_eimsbüttel = []

for cell in eimsbüttel.select("a"):
    stadtteil = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    boroughs_eimsbüttel.append({
        'Bezirk' : 'Eimsbüttel',
        'Stadtteil' : stadtteil,
        'URL' : url,
    })    
#boroughs_eimsbüttel

In [75]:
# Here the scraper only takes the last url – but why?
# First Person schould be Lieselotte Brandt in Eidelstedt

# with url = st['URL'] it is working here _ but not in the loop for the rows !!!

boroughs_eimsbüttel

for b in boroughs_eimsbüttel:
    url = b['URL'] 
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    b['Table'] = soup.select("table.wikitable")[0]
    b_table = b['Table']

boroughs_eimsbüttel[0]

{'Bezirk': 'Eimsbüttel',
 'Stadtteil': 'Eidelstedt',
 'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Eidelstedt',
 'Table': <table class="wikitable sortable toptextcells" style="width: 100%;">
 <tbody><tr>
 <th>Adresse
 </th>
 <th>Person(en)
 </th>
 <th class="unsortable">Inschrift
 </th>
 <th class="unsortable">Bilder
 </th>
 <th class="unsortable">Anmerkung
 </th></tr>
 <tr>
 <td data-sort-value="HolsteinerChaussee34">Holsteiner Chaussee 34<br/><span class="plainlinks-print" id="Lieselotte_Brandt" title="Koordinate"><a class="external text" href="https://geohack.toolforge.org/geohack.php?pagename=Liste_der_Stolpersteine_in_Hamburg-Eidelstedt&amp;language=de&amp;params=53.611230_N_9.905361_E_region:DE-HH_type:landmark&amp;title=Lieselotte+Brandt"><span typeof="mw:File"><span><img class="mw-file-element" data-file-height="256" data-file-width="256" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Erioll_world.svg/15px-Eriol

In [44]:
header_el = b_table.select("tr")[0]
columns = []
for cell in header_el.select("th"):
    columns.append(cell.text.strip())
columns.append('Bezirk')
columns.append('Stadtteil')
columns

['Adresse',
 'Person(en)',
 'Inschrift',
 'Bilder',
 'Anmerkung',
 'Bezirk',
 'Stadtteil']

In [76]:
## here it is wrong again - why? First Person should NOT be Hilde Falk in Försterweg 43

boroughs_eimsbüttel

for b in boroughs_eimsbüttel:
    row_els = b_table.select("tr")[1:]
    rows = []
    for row in row_els:
        cells = []
        for cell in row.select("td"):
            cells.append(cell.text.strip())
        cells.append(b['Bezirk'])    
        cells.append(b['Stadtteil'])
        rows.append(cells)   
rows[:2]        

[['Försterweg 43',
  'Hilde Falck',
  'Hier wohnte Hilde Falck Jg. 1924 deportiert 1942 Auschwitz ermordet',
  '',
  'Eintrag auf stolpersteine-hamburg.de',
  'Eimsbüttel',
  'Stellingen'],
 ['Försterweg 43',
  'Lina Falck geb. Heimann',
  'Hier wohnte Lina Falck geb. Heimann Jg. 1892 deportiert 1942 Auschwitz ermordet',
  '',
  'Eintrag auf stolpersteine-hamburg.de',
  'Eimsbüttel',
  'Stellingen']]

In [277]:
## count should be a lot more than 10

df = pd.DataFrame(rows, columns=columns)
df.describe()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil
count,10,10,10,10,10,10,10
unique,4,10,10,1,1,1,1
top,Försterweg 43,Hilde Falck,Hier wohnte Hilde Falck Jg. 1924 deportiert 19...,,Eintrag auf stolpersteine-hamburg.de,Eimsbüttel,Stellingen
freq,6,1,1,10,10,10,10


In [278]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for row in row_els:
    cells = []
    geo = []
    for cell in row.select("a"):
        cells.append(cell["href"])  
    for cell in cells:
        parsed_url = urlparse(cell)[4]
        geo.append(parsed_url)  
    for g in geo:
        if "pagename" in g:
            Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Stellingen&language=de&params=53.5785969_N_9.9303573_E_region:DE-HH_type:landmark&title=Hilde+Falck',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Stellingen&language=de&params=53.5785626_N_9.9303901_E_region:DE-HH_type:landmark&title=Lina+Falck+geb.+Heimann']

In [279]:
df['Geohack_url'] = Geohack

df['params'] = df['Geohack_url'].str.extract("params=(.*)region", expand=False)
df['lat'] = df['params'].str.extract("(.*)_N", expand=False)
df['lon'] = df['params'].str.extract("N_(.*)_E", expand=False)

df.head()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil,Geohack_url,params,lat,lon
0,Försterweg 43,Hilde Falck,Hier wohnte Hilde Falck Jg. 1924 deportiert 19...,,Eintrag auf stolpersteine-hamburg.de,Eimsbüttel,Stellingen,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5785969_N_9.9303573_E_,53.5785969,9.9303573
1,Försterweg 43,Lina Falck geb. Heimann,Hier wohnte Lina Falck geb. Heimann Jg. 1892 d...,,Eintrag auf stolpersteine-hamburg.de,Eimsbüttel,Stellingen,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5785626_N_9.9303901_E_,53.5785626,9.9303901
2,Försterweg 43,Ruth Falck,Hier wohnte Ruth Falck Jg. 1929 deportiert 194...,,Eintrag auf stolpersteine-hamburg.de,Eimsbüttel,Stellingen,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5785969_N_9.9303573_E_,53.5785969,9.9303573
3,Försterweg 43,Salomon Falck,Hier wohnte Salomon Falck Jg. 1897 deportiert ...,,Eintrag auf stolpersteine-hamburg.de,Eimsbüttel,Stellingen,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5785801_N_9.9303731_E_,53.5785801,9.9303731
4,Försterweg 43,Gerda Lazarus geb. Jacobs,Hier wohnte Gerda Lazarus geb. Jacobs Jg. 1900...,,Eintrag auf stolpersteine-hamburg.de,Eimsbüttel,Stellingen,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5785901_N_9.9304003_E_,53.5785901,9.9304003


## (4) Hamburg-Mitte – Incomplete, only shows last table

In [77]:
boroughs_hamburg_mitte = []

for cell in hamburg_mitte.select("a"):
    stadtteil = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    boroughs_hamburg_mitte.append({
        'Bezirk' : 'Hamburg-Mitte',
        'Stadtteil' : stadtteil,
        'URL' : url
    })    
    
boroughs_hamburg_mitte

[{'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Billbrook',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Billbrook'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Billstedt',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Billstedt'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Borgfelde',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Borgfelde'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Finkenwerder',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Finkenwerder'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Hamburg-Altstadt',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Altstadt'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Hamm',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Hamm'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Hammerbrook',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Ha

In [82]:
## Here it is working

boroughs_hamburg_mitte

for b in boroughs_hamburg_mitte:
    url = b['URL'] 
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    b['Table'] = soup.select("table.wikitable")[0]
    b_table = b['Table']
boroughs_hamburg_mitte[0]

{'Bezirk': 'Hamburg-Mitte',
 'Stadtteil': 'Billbrook',
 'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Billbrook',
 'Table': <table class="wikitable sortable" style="width: 100%;">
 <tbody><tr>
 <th>Adresse
 </th>
 <th>Person(en)
 </th>
 <th class="unsortable">Inschrift
 </th>
 <th class="unsortable">Bilder
 </th>
 <th class="unsortable">Anmerkung
 </th></tr>
 <tr style="vertical-align:top">
 <td data-sort-value="Billbrookdeich152">Billbrookdeich 152<br/><span class="plainlinks-print" id="Ljuba_Androssowa" title="Koordinate"><a class="external text" href="https://geohack.toolforge.org/geohack.php?pagename=Liste_der_Stolpersteine_in_Hamburg-Billbrook&amp;language=de&amp;params=53.536138_N_10.096823_E_region:DE-HH_type:landmark&amp;title=Ljuba+Androssowa"><span typeof="mw:File"><span><img class="mw-file-element" data-file-height="256" data-file-width="256" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Erioll_world.svg/15px

In [282]:
header_el = b_table.select("tr")[0]
columns = []
for cell in header_el.select("th"):
    columns.append(cell.text.strip())
columns.append('Bezirk')
columns.append('Stadtteil')
columns

['Adresse',
 'Person(en)',
 'Inschrift',
 'Bilder',
 'Anmerkung',
 'Bezirk',
 'Stadtteil']

In [84]:
## Here it is NOT working!!!

for b in boroughs_hamburg_mitte:
    b['Table'] = b_table
    row_els = b_table.select("tr")[1:]
    rows = []
    for row in row_els:
        cells = []
        for cell in row.select("td"):
            cells.append(cell.text.strip())
        cells.append(b['Bezirk'])    
        cells.append(b['Stadtteil'])
        rows.append(cells)   
rows[:2]        

[['Ilenbrook 14',
  'Bruno Schulz',
  'Hier wohnteBruno SchulzJg. 1912verhaftet 1939KZ FuhlsbüttelKZ Neuengammeermordet 8.2.1943',
  '',
  'Eintrag auf stolpersteine-hamburg.de',
  'Hamburg-Mitte',
  'Wilhelmsburg'],
 ['Industriestraße 160',
  'Edith Eggers geb. Libis',
  'Hier wohnteEdith Eggersgeb. LibisJg. 1908deportiert 1941Minsk???',
  '',
  'Eintrag auf stolpersteine-hamburg.deam Zugang zur Parkanlage am Ende der Karl-Kunert-Straße',
  'Hamburg-Mitte',
  'Wilhelmsburg']]

In [284]:
df = pd.DataFrame(rows, columns=columns)
df.describe()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil
count,22,22,22,22,22,22,22
unique,10,21,22,1,9,1,1
top,Rotenhäuser Straße 67,Hermine Baron geb. Löw,Hier wohnteBruno SchulzJg. 1912verhaftet 1939K...,,Eintrag auf stolpersteine-hamburg.de,Hamburg-Mitte,Wilhelmsburg
freq,4,2,1,22,10,22,22


In [285]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for row in row_els:
    cells = []
    geo = []
    for cell in row.select("a"):
        cells.append(cell["href"])  
    for cell in cells:
        parsed_url = urlparse(cell)[4]
        geo.append(parsed_url)  
    for g in geo:
        if "pagename" in g:
            Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Wilhelmsburg&language=de&params=53.5178221_N_9.9893026_E_region:DE-HH_type:landmark&title=Bruno+Schulz',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Wilhelmsburg&language=de&params=53.5192427_N_9.984024_E_region:DE-HH_type:landmark&title=Edith+Eggers+geb.+Libis']

In [286]:
df['Geohack_url'] = Geohack

df['params'] = df['Geohack_url'].str.extract("params=(.*)region", expand=False)
df['lat'] = df['params'].str.extract("(.*)_N", expand=False)
df['lon'] = df['params'].str.extract("N_(.*)_E", expand=False)

df.head()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil,Geohack_url,params,lat,lon
0,Ilenbrook 14,Bruno Schulz,Hier wohnteBruno SchulzJg. 1912verhaftet 1939K...,,Eintrag auf stolpersteine-hamburg.de,Hamburg-Mitte,Wilhelmsburg,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5178221_N_9.9893026_E_,53.5178221,9.9893026
1,Industriestraße 160,Edith Eggers geb. Libis,Hier wohnteEdith Eggersgeb. LibisJg. 1908depor...,,Eintrag auf stolpersteine-hamburg.deam Zugang ...,Hamburg-Mitte,Wilhelmsburg,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5192427_N_9.984024_E_,53.5192427,9.984024
2,Industriestraße 160,Frida Libis geb. Blumann,Hier wohnteFrida Libisgeb. BlumannJg. 1878depo...,,Eintrag auf stolpersteine-hamburg.deam Zugang ...,Hamburg-Mitte,Wilhelmsburg,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5192427_N_9.984024_E_,53.5192427,9.984024
3,Industriestraße 160,Siegmund Libis,Hier wohnteSiegmund LibisJg. 1878deportiert 19...,,Eintrag auf stolpersteine-hamburg.deam Zugang ...,Hamburg-Mitte,Wilhelmsburg,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5192427_N_9.984024_E_,53.5192427,9.984024
4,Kunertweg 5,Martin Furmanek,Hier wohnteMartin FurmanekJg. 1877hingerichtet...,,Eintrag auf stolpersteine-hamburg.de,Hamburg-Mitte,Wilhelmsburg,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5192451_N_9.9834458_E_,53.5192451,9.9834458


## (5) Hamburg-Nord - Looks complete

In [85]:
boroughs_hamburg_nord = []

for cell in hamburg_nord.select("a"):
    stadtteil = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    boroughs_hamburg_nord.append({
        'Bezirk' : 'Hamburg-Nord',
        'Stadtteil' : stadtteil,
        'URL' : url
    })    

In [86]:
boroughs_hamburg_nord

for b in boroughs_hamburg_nord:
    url = b['URL'] 
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    b['Table'] = soup.select("table.wikitable")[0]
    b_table = b['Table']
print(b_table.text[:100])



Adresse

Person(en)

Inschrift

Bilder

Anmerkung


Agnesstraße 2

Max Bernstein

Hier wohnte Max 


In [87]:
header_el = b_table.select("tr")[0]
columns = []
for cell in header_el.select("th"):
    columns.append(cell.text.strip())
columns.append('Bezirk')
columns.append('Stadtteil')
columns

['Adresse',
 'Person(en)',
 'Inschrift',
 'Bilder',
 'Anmerkung',
 'Bezirk',
 'Stadtteil']

In [88]:
for b in boroughs_hamburg_nord:
    b['Table'] = b_table
    row_els = b_table.select("tr")[1:]
    rows = []
    for row in row_els:
        cells = []
        for cell in row.select("td"):
            cells.append(cell.text.strip())
        cells.append(b['Bezirk'])    
        cells.append(b['Stadtteil'])
        rows.append(cells)   
rows[:2]        

[['Agnesstraße 2',
  'Max Bernstein',
  'Hier wohnte Max Bernstein Jg. 1896 verhaftet KZ Fuhlsbüttel KZ Neuengamme ermordet 14.10.1938',
  '',
  'Eintrag auf stolpersteine-hamburg.de',
  'Hamburg-Nord',
  'Winterhude'],
 ['Agnesstraße 30',
  'John Rittmeister',
  'Hier wohnte Dr. John Rittmeister Jg. 1898 inhaftiert hingerichtet 13.5.1943 Bln-Plötzensee',
  '',
  'Eintrag auf stolpersteine-hamburg.de',
  'Hamburg-Nord',
  'Winterhude']]

In [291]:
## count should be 328

df = pd.DataFrame(rows, columns=columns)
df.describe()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil
count,328,328,328,328,328,328,328
unique,194,325,327,1,43,1,1
top,Sierichstraße 153,Hermann Kaftal,Hier wohnte Gabriele Kaftal Jg. 1922 deportier...,,Eintrag auf stolpersteine-hamburg.de,Hamburg-Nord,Winterhude
freq,9,2,2,328,260,328,328


In [89]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for row in row_els:
    cells = []
    geo = []
    for cell in row.select("a"):
        cells.append(cell["href"])  
    for cell in cells:
        parsed_url = urlparse(cell)[4]
        geo.append(parsed_url)  
    for g in geo:
        if "pagename" in g:
            Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Winterhude&language=de&params=53.5810028_N_10.0004595_E_region:DE-HH_type:landmark&title=Max+Bernstein',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Winterhude&language=de&params=53.5826442_N_9.9987673_E_region:DE-HH_type:landmark&title=Dr.+John+Rittmeister']

In [90]:
## This was working before. Why is it not working anymore?

df['Geohack_url'] = Geohack

df['params'] = df['Geohack_url'].str.extract("params=(.*)region", expand=False)
df['lat'] = df['params'].str.extract("(.*)_N", expand=False)
df['lon'] = df['params'].str.extract("N_(.*)_E", expand=False)

df.head()

ValueError: Length of values (328) does not match length of index (34)

In [91]:
#df.to_csv('stolpersteine_Hamburg-Nord.csv', index = False)

## (6) Harburg – Incomplete

In [92]:
boroughs_harburg = []

for cell in harburg.select("a"):
    stadtteil = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    boroughs_harburg.append({
        'Bezirk' : 'Harburg',
        'Stadtteil' : stadtteil,
        'URL' : url
    })    

In [ ]:
## first person should be Rudolf Krukenbaum

boroughs_harburg

for b in boroughs_harburg:
    b['URL'] = b_url
    html = requests.get(b_url).text
    soup = BeautifulSoup(html)
   # b['Table'] = soup.select("table.wikitable")[0]
    b_table = soup.select("table.wikitable")[0]
    boroughs_harburg.append({
        'Table' : b_table
    })

#boroughs_harburg[0]
# This is correct, shows the first table

#print(b['Table'].text[:100])

print(b_table.text[:100]) 
#This is wrong

In [297]:
header_el = st_table.select("tr")[0]
columns = []
for cell in header_el.select("th"):
    columns.append(cell.text.strip())
columns.append('Bezirk')
columns.append('Stadtteil')
columns

['Adresse',
 'Person(en)',
 'Inschrift',
 'Bilder',
 'Anmerkung',
 'Bezirk',
 'Stadtteil']

In [298]:
for st in stadtteile_harburg:
    st['Table'] = st_table
    row_els = st_table.select("tr")[1:]
    rows = []
    for row in row_els:
        cells = []
        for cell in row.select("td"):
            cells.append(cell.text.strip())
        cells.append(st['Bezirk'])    
        cells.append(st['Stadtteil'])
        rows.append(cells)   
rows[:2]        

[['Am Frankenberg 8',
  'Liselott Kreidelmeyer',
  'Hier wohnte Liselott Kreidelmeyer Jg. 1923 eingewiesen 1938 Alsterdorfer Anstalten ‚verlegt‘ 1943 Heilanstalt Am Steinhof/Wien ermordet 27.8.1943',
  '',
  'Eintrag auf stolpersteine-hamburg.de',
  'Harburg',
  'Wilstorf'],
 ['Am Frankenberg 22',
  'Heinrich Coors',
  'Hier wohnte Heinrich Coors Jg. 1911 im Widerstand verhaftet 1942 Haft in Hamburg 1944 Neuengamme tot 3.5.1945 Neustädter Bucht',
  '',
  'Eintrag auf stolpersteine-hamburg.de',
  'Harburg',
  'Wilstorf']]

In [299]:
df = pd.DataFrame(rows, columns=columns)
df.describe()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil
count,21,21,21,21,21,21,21
unique,20,21,21,1,3,1,1
top,Kapellenweg 15,Liselott Kreidelmeyer,Hier wohnte Liselott Kreidelmeyer Jg. 1923 ein...,,Eintrag auf stolpersteine-hamburg.de,Harburg,Wilstorf
freq,2,1,1,21,18,21,21


In [300]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for row in row_els:
    cells = []
    geo = []
    for cell in row.select("a"):
        cells.append(cell["href"])  
    for cell in cells:
        parsed_url = urlparse(cell)[4]
        geo.append(parsed_url)  
    for g in geo:
        if "pagename" in g:
            Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Wilstorf&language=de&params=53.436616_N_9.984773_E_region:DE-HH_type:landmark&title=Liselott+Kreidelmeyer',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Wilstorf&language=de&params=53.436909_N_9.983737_E_region:DE-HH_type:landmark&title=Heinrich+Coors']

In [301]:
df['Geohack_url'] = Geohack

df['params'] = df['Geohack_url'].str.extract("params=(.*)region", expand=False)
df['lat'] = df['params'].str.extract("(.*)_N", expand=False)
df['lon'] = df['params'].str.extract("N_(.*)_E", expand=False)

df.head()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil,Geohack_url,params,lat,lon
0,Am Frankenberg 8,Liselott Kreidelmeyer,Hier wohnte Liselott Kreidelmeyer Jg. 1923 ein...,,Eintrag auf stolpersteine-hamburg.de,Harburg,Wilstorf,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.436616_N_9.984773_E_,53.436616,9.984773
1,Am Frankenberg 22,Heinrich Coors,Hier wohnte Heinrich Coors Jg. 1911 im Widerst...,,Eintrag auf stolpersteine-hamburg.de,Harburg,Wilstorf,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.436909_N_9.983737_E_,53.436909,9.983737
2,Am Mühlenfeld 107,Karl Kock,Hier wohnte Karl Kock Jg. 1908 Widerstandskämp...,,Eintrag auf stolpersteine-hamburg.deEin weiter...,Harburg,Wilstorf,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.443200_N_9.982579_E_,53.443200,9.982579
3,Anzengruberstraße 7,Karl Suderburg,Hier wohnte Karl Suderburg Jg. 1910 eingewiese...,,Eintrag auf stolpersteine-hamburg.de,Harburg,Wilstorf,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.450333_N_9.989277_E_,53.450333,9.989277
4,Besselstraße 4,Martha Hermann geb. Müller,Hier wohnte Martha Hermann geb. MüllerJg. 1897...,,Eintrag auf stolpersteine-hamburg.de,Harburg,Wilstorf,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.443092_N_9.999686_E_,53.443092,9.999686


## (7) Wandsbek – Incomplete

In [302]:
stadtteile_wandsbek = []

for cell in wandsbek.select("a"):
    stadtteil = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    stadtteile_wandsbek.append({
        'Bezirk' : 'Wandsbek',
        'Stadtteil' : stadtteil,
        'URL' : url
    })    

In [303]:
stadtteile_wandsbek

for st in stadtteile_wandsbek:
    url = st['URL'] 
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    st['Table'] = soup.select("table.wikitable")[0]
    st_table = st['Table']
print(st_table.text[:100])



Adresse

Person(en)

Inschrift

Bilder

Anmerkung


Hasselwisch 5 

Georg Mewes

Hier wohnteGeorg 


In [304]:
header_el = st_table.select("tr")[0]
columns = []
for cell in header_el.select("th"):
    columns.append(cell.text.strip())
columns.append('Bezirk')
columns.append('Stadtteil')
columns

['Adresse',
 'Person(en)',
 'Inschrift',
 'Bilder',
 'Anmerkung',
 'Bezirk',
 'Stadtteil']

In [305]:
for st in stadtteile_wandsbek:
    st['Table'] = st_table
    row_els = st_table.select("tr")[1:]
    rows = []
    for row in row_els:
        cells = []
        for cell in row.select("td"):
            cells.append(cell.text.strip())
        cells.append(st['Bezirk'])    
        cells.append(st['Stadtteil'])
        rows.append(cells)   
rows[:2]        

[['Hasselwisch 5',
  'Georg Mewes',
  'Hier wohnteGeorg MewesJg. 1909verhaftet 1943‚Spionageverdacht‘KZ FuhlsbüttelBuchenwaldermordet 2.1.1944',
  '',
  'Eintrag auf stolpersteine-hamburg.de',
  'Wandsbek',
  'Wohldorf-Ohlstedt'],
 ['Lottbeker Weg 13',
  'Dorothea Gatzenmeyer geb. Jürs',
  'Hier wohnteDorotheaGatzenmeyergeb. JürsJg. 1907im Widerstand/SPDFlucht 1937Dänemark',
  '',
  'Eintrag auf stolpersteine-hamburg.de',
  'Wandsbek',
  'Wohldorf-Ohlstedt']]

In [306]:
df = pd.DataFrame(rows, columns=columns)
df.describe()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil
count,5,5,5,5,5,5,5
unique,4,5,5,1,3,1,1
top,Lottbeker Weg 13,Georg Mewes,Hier wohnteGeorg MewesJg. 1909verhaftet 1943‚S...,,Eintrag auf stolpersteine-hamburg.de,Wandsbek,Wohldorf-Ohlstedt
freq,2,1,1,5,3,5,5


In [307]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for row in row_els:
    cells = []
    geo = []
    for cell in row.select("a"):
        cells.append(cell["href"])  
    for cell in cells:
        parsed_url = urlparse(cell)[4]
        geo.append(parsed_url)  
    for g in geo:
        if "pagename" in g:
            Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Wohldorf-Ohlstedt&language=de&params=53.6967839_N_10.1432031_E_region:DE-HH_type:landmark&title=Georg+Mewes',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Wohldorf-Ohlstedt&language=de&params=53.690128_N_10.142705_E_region:DE-HH_type:landmark&title=Dorothea+Gatzenmeyer+geb.+J%C3%BCrs']

In [308]:
df['Geohack_url'] = Geohack

df['params'] = df['Geohack_url'].str.extract("params=(.*)region", expand=False)
df['lat'] = df['params'].str.extract("(.*)_N", expand=False)
df['lon'] = df['params'].str.extract("N_(.*)_E", expand=False)

df.head()

,Adresse,Person(en),Inschrift,Bilder,Anmerkung,Bezirk,Stadtteil,Geohack_url,params,lat,lon
0,Hasselwisch 5,Georg Mewes,Hier wohnteGeorg MewesJg. 1909verhaftet 1943‚S...,,Eintrag auf stolpersteine-hamburg.de,Wandsbek,Wohldorf-Ohlstedt,pagename=Liste_der_Stolpersteine_in_Hamburg-Wo...,53.6967839_N_10.1432031_E_,53.6967839,10.1432031
1,Lottbeker Weg 13,Dorothea Gatzenmeyer geb. Jürs,Hier wohnteDorotheaGatzenmeyergeb. JürsJg. 190...,,Eintrag auf stolpersteine-hamburg.de,Wandsbek,Wohldorf-Ohlstedt,pagename=Liste_der_Stolpersteine_in_Hamburg-Wo...,53.690128_N_10.142705_E_,53.690128,10.142705
2,Lottbeker Weg 13,Louis Gatzenmeyer,Hier wohnteLouisGatzenmeyerJg. 1905im Widersta...,,Eintrag auf stolpersteine-hamburg.de,Wandsbek,Wohldorf-Ohlstedt,pagename=Liste_der_Stolpersteine_in_Hamburg-Wo...,53.690128_N_10.142705_E_,53.690128,10.142705
3,Lottbeker Weg 24,Chaja Rywa Balck geb. Skop,Hier wohnteChaja Rywa Balckgeb. SkopJg. 1895de...,,Eintrag auf stolpersteine-hamburg.deStolperste...,Wandsbek,Wohldorf-Ohlstedt,pagename=Liste_der_Stolpersteine_in_Hamburg-Wo...,53.6892268_N_10.1426552_E_,53.6892268,10.1426552
4,Schleusenredder 23,Alwin Cäsar Gerson,Hier wohnteDr. Alwin CäsarGersonJg. 1866deport...,,Eintrag auf stolpersteine-hamburg.deEin weiter...,Wandsbek,Wohldorf-Ohlstedt,pagename=Liste_der_Stolpersteine_in_Hamburg-Wo...,53.7058046_N_10.1252442_E_,53.7058046,10.1252442


# Analysis

#### Get year of birth from inscription ("Inschrift")

In [309]:
df['Jahrgang'] = df['Inschrift'].str.extract("Jg. (\d*)", expand=False).astype(float)

#### Get year of deportation from inscription

In [310]:
df['deportiert'] = df['Inschrift'].str.extract("deportiert (\d*)", expand=False).astype(float)

#### Get age when deported

In [311]:
df['age_when_deported']= df(df['deportiert'] - df['Jahrgang'])

TypeError: 'DataFrame' object is not callable

#### Get year of "Polenaktion"

In [ ]:
df['Polenaktion'] = df['Inschrift'].str.extract("'Polenaktion' (\d*)", expand=False)

#### Get murderd ("ermordet")

In [ ]:
df['ermordet'] = df['Inschrift'].str.contains("ermordet")

#### Get Auschwitz

In [ ]:
df['Auschwitz'] = df['Inschrift'].str.contains("Auschwitz")

In [ ]:
df.head()

In [ ]:
#page_title = []

#for st in stadtteile:
#        html = requests.get(url).text
#        soup = BeautifulSoup(html)
#        title = soup.select("firstHeading")
#    page_title.append(title)
        print(title.text[:1])
    
#df['Stadtteil'] = stadtteil